# CoSen: Cost Sensitive CNN loss

In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from costsensitive.losses import CostSensitiveLoss, CostSensitiveRegularizedLoss

Basic usage:

In [2]:
n_classes = 3
criterion = CostSensitiveLoss(n_classes)

M = np.array([[0,0,0], [0,0,0], [10, 0, 0]], dtype=float)
M /= M.max()
criterion.M = torch.from_numpy(M)

label = torch.tensor([2]).unsqueeze(0)
logits_bad = torch.tensor([10., 0., 0.]).unsqueeze(0)
print('Bad logits:', criterion(logits_bad, label))
logits_ok = torch.Tensor([0., 0., 10.]).unsqueeze(0)
print('Good logits:', criterion(logits_ok, label))

Bad logits: tensor(0.9999, dtype=torch.float64)
Good logits: tensor(4.5396e-05, dtype=torch.float64)


Mixed as a regularizer Cross Entropy Loss Function:

In [24]:
n_classes = 3 
base_loss = 'ce'
lambd = 10
cs_regularized_criterion = CostSensitiveRegularizedLoss(n_classes=n_classes, base_loss=base_loss, lambd=lambd)
labels = torch.tensor([2])
worse_logits = torch.tensor([10.0, 0.0, 0.0]).unsqueeze(0)
better_logits = torch.tensor([0.0, 10.0, 0.0]).unsqueeze(0)
perfect_logits = torch.tensor([0.0, 0.0, 10.0]).unsqueeze(0)
print('Cost Sensitive-Regularized CE loss:')
print('Worst prediction loss = {:.4f}'.format(cs_regularized_criterion(worse_logits, labels)))
print('Better prediction loss = {:.4f}'.format(cs_regularized_criterion(better_logits, labels)))
print('Perfect prediction loss = {:.4f}'.format(cs_regularized_criterion(perfect_logits, labels)))

Cost Sensitive-Regularized CE loss:
Worst prediction loss = 19.9993
Better prediction loss = 12.5003
Perfect prediction loss = 0.0007


And CE with Label-Smoothing (LS):

In [25]:
base_loss = 'ls'
cs_regularized_criterion = CostSensitiveRegularizedLoss(n_classes=n_classes, base_loss=base_loss, lambd=lambd)
print('Cost Sensitive-Regularized CE loss with Label Smoothing:')
print('Worst prediction loss = {:.4f}'.format(cs_regularized_criterion(worse_logits, labels)))
print('Better prediction loss = {:.4f}'.format(cs_regularized_criterion(better_logits, labels)))
print('Perfect prediction loss = {:.4f}'.format(cs_regularized_criterion(perfect_logits, labels)))

Cost Sensitive-Regularized CE loss with Label Smoothing:
Worst prediction loss = 19.6660
Better prediction loss = 12.1670
Perfect prediction loss = 0.6673


Finally, CE + Gaussian LS:

In [26]:
base_loss = 'gls'
cs_regularized_criterion = CostSensitiveRegularizedLoss(n_classes=n_classes, base_loss=base_loss, lambd=lambd)
print('Cost Sensitive-Regularized CE loss with Label Smoothing:')
print('Worst prediction loss = {:.4f}'.format(cs_regularized_criterion(worse_logits, labels)))
print('Better prediction loss = {:.4f}'.format(cs_regularized_criterion(better_logits, labels)))
print('Perfect prediction loss = {:.4f}'.format(cs_regularized_criterion(perfect_logits, labels)))

Cost Sensitive-Regularized CE loss with Label Smoothing:
Worst prediction loss = 11.0818
Better prediction loss = 10.4818
Perfect prediction loss = 9.0593
